In [16]:
import pandas
from time import time
from sklearn.metrics import accuracy_score
from collections import Counter
from sklearn.metrics import confusion_matrix
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import GaussianNB
col_names = ["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]
kdd_data_10percent = pandas.read_csv("data\\dataset2", header=None, names = col_names)
kdd_data_10percent.describe()


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
count,147838.000000,1.478380e+05,1.478380e+05,147838.000000,147838.000000,147838.000000,147838.000000,147838.000000,147838.000000,147838.000000,...,147838.000000,147838.000000,147838.000000,147838.000000,147838.000000,147838.000000,147838.000000,147838.000000,147838.000000,147838.000000
mean,123.138990,5.232784e+04,6.706823e+03,0.000467,0.045205,0.001339,0.756152,0.019312,0.360232,0.119509,...,195.740101,152.716372,0.731881,0.103555,0.295514,0.043844,0.115120,0.113749,0.095216,0.092336
std,1563.874919,4.057662e+06,1.649974e+05,0.021599,0.350530,0.041915,3.870293,0.144336,0.480069,1.240077,...,101.219335,114.850269,0.413515,0.269012,0.439489,0.149056,0.307468,0.315510,0.267315,0.281674
min,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.100000e+01,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,155.000000,13.000000,0.300000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,1.050000e+02,4.300000e+01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,255.000000,250.000000,1.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,3.340000e+02,1.740000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,255.000000,255.000000,1.000000,0.030000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,58329.000000,6.933756e+08,5.155468e+06,1.000000,3.000000,3.000000,30.000000,5.000000,1.000000,238.000000,...,255.000000,255.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [17]:
kdd_data_10percent['label'].value_counts()

snmpgetattack.      23223
normal.             20000
smurf.              15000
neptune.            15000
guess_passwd.       13631
warezclient.        10200
satan.               7945
back.                7700
snmpguess.           7218
ipsweep.             6235
portsweep.           5200
warezmaster.         5006
buffer_overflow.     3666
teardrop.            1994
rootkit.             1239
nmap.                1155
loadmodule.          1086
pod.                  789
httptunnel.           474
perl.                 366
multihop.             124
imap.                 123
ftp_write.             89
land.                  69
sendmail.              51
named.                 51
ps.                    48
phf.                   46
xterm.                 39
xlock.                 27
spy.                   20
xsnoop.                12
sqlattack.              6
worm.                   6
Name: label, dtype: int64

In [18]:

num_features = [
    "duration","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate"
]
features = kdd_data_10percent[num_features].astype(float)
features.describe()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate
count,147838.000000,1.478380e+05,1.478380e+05,147838.000000,147838.000000,147838.000000,147838.000000,147838.000000,147838.000000,147838.000000,...,147838.000000,147838.000000,147838.000000,147838.000000,147838.000000,147838.000000,147838.000000,147838.000000,147838.000000,147838.000000
mean,123.138990,5.232784e+04,6.706823e+03,0.000467,0.045205,0.001339,0.756152,0.019312,0.360232,0.119509,...,195.740101,152.716372,0.731881,0.103555,0.295514,0.043844,0.115120,0.113749,0.095216,0.092336
std,1563.874919,4.057662e+06,1.649974e+05,0.021599,0.350530,0.041915,3.870293,0.144336,0.480069,1.240077,...,101.219335,114.850269,0.413515,0.269012,0.439489,0.149056,0.307468,0.315510,0.267315,0.281674
min,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.100000e+01,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,155.000000,13.000000,0.300000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,1.050000e+02,4.300000e+01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,255.000000,250.000000,1.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,3.340000e+02,1.740000e+02,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,255.000000,255.000000,1.000000,0.030000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,58329.000000,6.933756e+08,5.155468e+06,1.000000,3.000000,3.000000,30.000000,5.000000,1.000000,238.000000,...,255.000000,255.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [19]:

train_labels = kdd_data_10percent['label'].copy()
labels = []


for label in train_labels:
    if label in ['nmap.', 'portsweep.', 'ipsweep.', 'satan.','mscan.','saint.']:
        label = 'probing'
    elif label in ['land.','pod.','teardrop.','back.','neptune.','smurf.','apache2.','mailbomb.','processtable.','udpstorm.']:
        label = 'dos'
    elif label in ['spy.', 'phf.', 'multihop.', 'ftp_write.', 'imap.', 'warezmaster.', 'warezclient.', 'guess_passwd.','sendmail.','named.','snmpgetattack.','snmpguess.','xlock.','xsnoop.','worm.']:
        label = 'r2l'
    elif label in ['buffer_overflow.', 'rootkit.','loadmodule.','perl.','httptunnel.','ps.','sqlattack.','xterm.']:
        label = 'u2r'
    labels.append(label)
labels = pandas.Series( (v[0] for v in labels) )
print(type(labels))
labels.value_counts()

<class 'pandas.core.series.Series'>


r    59827
d    40552
p    20535
n    20000
u     6924
dtype: int64

In [20]:
features.shape

(147838, 38)

In [21]:

t0 = time()

gnb = GaussianNB().fit(features,labels)
tt = time()-t0
print ( "Classifier trained in {} seconds".format(round(tt,3)))

Classifier trained in 0.612 seconds


In [22]:
kdd_data_corrected = pandas.read_csv("corrected\\corrected", header=None, names = col_names)
kdd_data_corrected['label'].value_counts()

smurf.              164091
normal.              60593
neptune.             58001
snmpgetattack.        7741
mailbomb.             5000
guess_passwd.         4367
snmpguess.            2406
satan.                1633
warezmaster.          1602
back.                 1098
mscan.                1053
apache2.               794
processtable.          759
saint.                 736
portsweep.             354
ipsweep.               306
httptunnel.            158
pod.                    87
nmap.                   84
buffer_overflow.        22
multihop.               18
sendmail.               17
named.                  17
ps.                     16
rootkit.                13
xterm.                  13
teardrop.               12
land.                    9
xlock.                   9
xsnoop.                  4
ftp_write.               3
worm.                    2
loadmodule.              2
phf.                     2
sqlattack.               2
perl.                    2
udpstorm.                2
i

In [23]:
train_labels = kdd_data_corrected['label'].copy()
labels_test = []


for label in train_labels:
    if label in ['nmap.', 'portsweep.', 'ipsweep.', 'satan.','mscan.','saint.']:
        label = 'probing'
    elif label in ['land.','pod.','teardrop.','back.','neptune.','smurf.','apache2.','mailbomb.','processtable.','udpstorm.']:
        label = 'dos'
    elif label in ['spy.', 'phf.', 'multihop.', 'ftp_write.', 'imap.', 'warezmaster.', 'warezclient.', 'guess_passwd.','sendmail.','named.','snmpgetattack.','snmpguess.','xlock.','xsnoop.','worm.']:
        label = 'r2l'
    elif label in ['buffer_overflow.', 'rootkit.','loadmodule.','perl.','httptunnel.','ps.','sqlattack.','xterm.']:
        label = 'u2r'
    labels_test.append(label)
labels_test = pandas.Series( (v[0] for v in labels) )

In [24]:
#kdd_data_corrected['label'][kdd_data_corrected['label']!='normal.'] = 'attack.'
kdd_data_corrected['label']=labels_test[:]
kdd_data_corrected['label'].value_counts()


r    59827
d    40552
p    20535
n    20000
u     6924
Name: label, dtype: int64

In [25]:

kdd_data_corrected[num_features] = kdd_data_corrected[num_features].astype(float)
#kdd_data_corrected[num_features].apply(lambda x: MinMaxScaler().fit_transform(x))

In [26]:
features_train, features_test, labels_train, labels_test = train_test_split(
    kdd_data_corrected[num_features], 
    kdd_data_corrected['label'],  
    random_state=0)

In [27]:

t0 = time()
pred = gnb.predict(features_test)

tt = time() - t0
print ("Predicted in {} seconds".format(round(tt,3)))

Predicted in 0.552 seconds


In [28]:

#print(type(labels))
print(labels_test.value_counts())
i=(1)
q=Counter(pred)
print(q)

acc = accuracy_score(pred, labels_test)
print ("R squared is {}.".format(round(acc,4)))

r    15074
d    10160
p     5127
n     4987
u     1782
Name: label, dtype: int64
Counter({'d': 73396, 'u': 3054, 'n': 939, 'p': 341, 'r': 28})


ValueError: Classification metrics can't handle a mix of multiclass and unknown targets

In [ ]:

con = confusion_matrix(pred,labels_test)
print(pred[0])
print(con)
#d=0
#n=1
#p=2
#r=3
#u=4

In [ ]:

par=np.zeros((4,5))

for i in range (0,5):
    par[0,i]=con[i,i]
    
for i in range (0,5):
    for j in range (0,5):
        if i!=j:
            par[1,i]=par[1,i]+con[i,j]

for i in range (0,5):
    for j in range (0,5):
        if i!=j:
            par[2,i]=par[2,i]+con[j,i]        

for i in range (0,5):
    for j in range (0,5):
        for k in range (0,5):
            if j!=i and k!=i:
                par[3,i]=par[3,i]+con[i,j]

                
#2D matrix par
#tp-0
#fn-1
#fp-2
#tn-3
#2D matrix ratio
#tpr-4
#fpr-5
#precision-6
#accuracy-7
ratio=np.zeros((4,5))
for i in range (0,5):
    ratio[0,i]=(par[0,i]/(par[0,i]+par[1,i]))*100
    
for i in range (0,5):
    ratio[1,i]=(par[2,i]/(par[2,i]+par[3,i]))*100
for i in range (0,5):
    ratio[2,i]=par[0,i]/(par[0,i]+par[2,i])
for i in range (0,5):
    ratio[3,i]=((par[0,i]+par[3,i])/(par[0,i]+par[1,i]+par[2,i]+par[3,i]))*100


np.set_printoptions(formatter={'float_kind':'{:f}'.format})       
print('Parameters')
print(par.astype(int))
print('Parameter ratio')
print(ratio)